In [1]:
import numpy as np
import sympy as sp
from typing import List, Tuple, Dict, Any, Set
from scipy.stats import levy
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

class all_parameter_generation:
    """
    Generate state transitions and random parameters (a, b, c, enzyme) for an n-site phosphorylation model.

    Args:
        n: number of sites (int)
        distribution: distribution name ("gamma" supported)
        params: parameters for the distribution (for gamma: [shape, scale])
        verbose: if True, prints transitions and matrices
    """
    def __init__(self, n: int, reaction_types: str, distribution: str, distribution_paramaters: List[float], verbose: bool = False):
        self.n = n
        self.num_states = 2 ** n
        self.distribution = distribution
        self.params = distribution_paramaters
        self.reaction_types = reaction_types
        self.verbose = verbose
        self.rng = np.random.default_rng()
        
    @staticmethod
    def padded_binary(i: int, n: int) -> str:
        return bin(i)[2:].zfill(n)

    @staticmethod
    def binary_string_to_array(string: str) -> np.ndarray:
        return np.array([int(i) for i in string], dtype=int)

    def calculate_valid_transitions(self) -> Tuple[List[List[Any]], List[List[Any]]]:
        """
        Returns:
            valid_X_reactions: list of [state_i_str, state_j_str, i, j, "E"]
            valid_Y_reactions: list of [state_i_str, state_j_str, i, j, "F"]
        """
        all_states = [self.padded_binary(i, self.n) for i in range(self.num_states)]

        valid_difference_vectors: Set[Tuple[int, ...]] = set()
        valid_X_reactions: List[List[Any]] = []
        valid_Y_reactions: List[List[Any]] = []

        for i in range(self.num_states):
            arr_i = self.binary_string_to_array(all_states[i])
            for j in range(self.num_states):
                if i == j:
                    continue
                arr_j = self.binary_string_to_array(all_states[j])
                diff = arr_j - arr_i
                # if self.reaction_types == "distributive":
                    
                hamming_weight = np.sum(np.abs(diff))

                if hamming_weight == 1:
                    # +1 -> phosphorylation (E), -1 -> dephosphorylation (F)
                    element = "E" if np.any(diff == 1) else "F"
                    if element == "E":
                        if self.verbose:
                            print(f"{all_states[i]} --> {all_states[j]} (E), {i}, {j}")
                        valid_X_reactions.append([all_states[i], all_states[j], i, j, element])
                    else:
                        if self.verbose:
                            print(f"{all_states[i]} --> {all_states[j]} (F), {i}, {j}")
                        valid_Y_reactions.append([all_states[i], all_states[j], i, j, element])
                    valid_difference_vectors.add(tuple(diff))

        return valid_X_reactions, valid_Y_reactions
    
    def alpha_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray,
                                            Dict[int, List[int]], Dict[int, List[int]],
                                            Dict[int, List[int]], Dict[int, List[int]]]:
        
        valid_X_reactions, valid_Y_reactions = self.calculate_valid_transitions()

        shape, scale = self.params

        alpha_matrix = np.zeros((self.num_states, self.num_states))

        for _, _, i, j, _ in valid_X_reactions:

            alpha_matrix[i][j] = self.rng.gamma(shape, scale)

        return alpha_matrix

    def beta_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray,
                                            Dict[int, List[int]], Dict[int, List[int]],
                                            Dict[int, List[int]], Dict[int, List[int]]]:
        
        valid_X_reactions, valid_Y_reactions = self.calculate_valid_transitions()

        shape, scale = self.params
        beta_matrix = np.zeros((self.num_states, self.num_states))
        
        for _, _, i, j, _ in valid_Y_reactions:

            beta_matrix[i][j] = self.rng.gamma(shape, scale)

        return beta_matrix
    
    def k_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray]:
        # if self.distribution != "gamma":
        #     raise NotImplementedError("Only 'gamma' distribution implemented for a_parameter_generation")
        shape, scale = self.params
        if self.distribution == "gamma":
            k_positive_rates = self.rng.gamma(shape, scale, self.num_states - 1)
            k_negative_rates = self.rng.gamma(shape, scale, self.num_states - 1)
        if self.distribution == "levy":
            k_positive_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)
            k_negative_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)
        # k_positive_rates[-1] = 0
        # k_negative_rates[-1] = 0
        
        return k_positive_rates, k_negative_rates

    def p_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray]:
        
        # if self.distribution != "gamma":
        #     raise NotImplementedError("Only 'gamma' distribution implemented for b_parameter_generation")
        shape, scale = self.params
        if self.distribution == "gamma":
            p_positive_rates = self.rng.gamma(shape, scale, self.num_states - 1)
            p_negative_rates = self.rng.gamma(shape, scale, self.num_states - 1)
        if self.distribution == "levy":
            p_positive_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)
            p_negative_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)
        # p_positive_rates[0] = 0
        # p_negative_rates[0] = 0

        return p_positive_rates, p_negative_rates
    

In [2]:
from sympy import solve
# from sympy.abc import A0, A1, B0, C1, X, Y
X = sp.Symbol('X'); Y = sp.Symbol('Y')
A = sp.IndexedBase('A')
B = sp.IndexedBase('B')
C = sp.IndexedBase('C')
solutions = solve([-X*A[0] + B[0] + C[1], -Y*A[1] + C[1] + B[0], X*A[0] - 2*B[0], Y*A[1] - 2*C[1], -X*A[0] + 2*B[0], -Y*A[1] + 2*C[1]], [A[0], A[1], B[0], C[1], X, Y], dict=True)
solutions[0][A[0]]


Y*A[1]/X

In [3]:
solutions[0][B[0]]

Y*A[1]/2

In [4]:
solutions[0][C[1]]

Y*A[1]/2

In [5]:
import sympy as sp

def build_phos_odes(n, prefix=""):
    """
    Build symbolic ODEs for the phosphorylation system for given n (N = 2**n).
    Returns a dict with keys:
      N, a, b, c, x, y,
      Kp, Km, Pp, Pm, Alpha, Beta, DAlpha, DBeta,
      adot, bdot, cdot, xdot, ydot,
      ones (vector of ones)
    """
    N = 2**n
    a_syms = sp.symbols([f"{prefix}a_{i}" for i in range(N)])
    b_syms = sp.symbols([f"{prefix}b_{i}" for i in range(N)])
    c_syms = sp.symbols([f"{prefix}c_{i}" for i in range(N)])
    x_sym, y_sym = sp.symbols(f"{prefix}x {prefix}y")

    a = sp.Matrix(N, 1, lambda i,j: a_syms[i])
    b = sp.Matrix(N, 1, lambda i,j: b_syms[i])
    b[-1] = 0
    print(b)
    c = sp.Matrix(N, 1, lambda i,j: c_syms[i])
    c[0] = 0

    print(c)

    x = x_sym
    y = y_sym

    kplus = sp.symbols([f"{prefix}k^+_{i}" for i in range(N)])
    kminus = sp.symbols([f"{prefix}k^-_{i}" for i in range(N)])
    pplus = sp.symbols([f"{prefix}p^+_{i}" for i in range(N)])
    pminus = sp.symbols([f"{prefix}p^-_{i}" for i in range(N)])

    Kp = sp.diag(*kplus)
    Km = sp.diag(*kminus)
    Pp = sp.diag(*pplus)
    Pm = sp.diag(*pminus)

    valid_X_reactions, valid_Y_reactions = calculate_valid_transitions(n)

    # compute allowed transitions
    valid_X, valid_Y = calculate_valid_transitions(n)
    
    # Build Alpha and Beta with symbols only at allowed (j,k)
    Alpha = sp.zeros(N, N)
    Beta  = sp.zeros(N, N)
    # create symbols for allowed entries
    for (_, _, j,k, _) in valid_X:
        Alpha[j,k] = sp.symbols(f"{prefix}alpha_{j}_{k}")
    for (_, _, j,k, _) in valid_Y:
        Beta[j,k] = sp.symbols(f"{prefix}beta_{j}_{k}")
      
    # Set last column and row of Alpha to zero (no transitions from or to fully phosphorylated state)
    # Alpha[:, N-1] = sp.zeros(N, 1)
    # Alpha[N-1, :] = sp.zeros(1, N)
    # # Set first column and row of Beta to zero (no transitions from or to fully de
    # Beta[:, 0] = sp.zeros(N, 1)
    # Beta[0, :] = sp.zeros(1, N)
    print("Alpha matrix:")
    sp.pprint(Alpha)
    print("Beta matrix:")
    sp.pprint(Beta)
    ones = sp.Matrix([1]*N)
    Alpha_row_sums = Alpha * ones
    Beta_row_sums = Beta * ones
    DAlpha = sp.diag(*[Alpha_row_sums[i,0] for i in range(N)])
    DBeta  = sp.diag(*[Beta_row_sums[i,0]  for i in range(N)])

    # a_adjusted_for_b_dot = a.copy(); a_adjusted_for_b_dot[-1] = 0
    # a_adjusted_for_c_dot = a.copy(); a_adjusted_for_c_dot[0] = 0

    # a_adjusted_for_x_dot = a.copy(); a_adjusted_for_x_dot[-1] = 0
    # a_adjusted_for_y_dot = a.copy(); a_adjusted_for_y_dot[0] = 0

    adot = Km * b + Pm * c + Alpha.T * b + Beta.T * c - x * (Kp * a) - y * (Pp * a)
    bdot = x * (Kp * a) - Km * b - DAlpha * b
    cdot = y * (Pp * a) - Pm * c - DBeta * c

    kplus_vec = sp.Matrix(kplus)
    pplus_vec = sp.Matrix(pplus)

    xdot = - x * (kplus_vec.T * a)[0] + (ones.T * (Km + DAlpha) * b)[0]
    ydot = - y * (pplus_vec.T * a)[0] + (ones.T * (Pm + DBeta) * c)[0]
    
    return {
        "N": N,
        "a": a, "b": b, "c": c, "x": x, "y": y,
        "Kp": Kp, "Km": Km, "Pp": Pp, "Pm": Pm,
        "Alpha": Alpha, "Beta": Beta,
        "DAlpha": DAlpha, "DBeta": DBeta,
        "ones": ones,
        "adot": sp.simplify(adot), "bdot": sp.simplify(bdot), "cdot": sp.simplify(cdot),
        "xdot": sp.simplify(xdot), "ydot": sp.simplify(ydot)
    }

# Example: build symbolic ODEs for n=2 (N=4)
n = 1
odes = build_phos_odes(n)
odes["adot"][0]
# sp.pprint(odes["adot"])
# sp.pprint(odes["bdot"])
# sp.pprint(odes["cdot"])
# sp.pprint(odes["xdot"])
# sp.pprint(odes["ydot"])


Matrix([[b_0], [0]])
Matrix([[0], [c_1]])


NameError: name 'calculate_valid_transitions' is not defined

In [6]:
odes["bdot"][0]


NameError: name 'odes' is not defined

In [7]:
odes["bdot"][1]


NameError: name 'odes' is not defined

In [8]:
odes["cdot"][1]

NameError: name 'odes' is not defined

In [9]:
odes["xdot"]

NameError: name 'odes' is not defined

In [10]:
odes["ydot"]

NameError: name 'odes' is not defined

# BEST SYMBOLIC ODE GENERATOR:

In [11]:
import sympy as sp
def padded_binary(i: int, n: int) -> str:
    return bin(i)[2:].zfill(n)

# Replace this method in your class
@staticmethod
def binary_string_to_array(string: str) -> np.ndarray:
    # use a comprehension so we don't rely on the builtin `list` name being callable
    return np.array([int(ch) for ch in string], dtype=int)

def calculate_valid_transitions(n: int):
    num_states = 2**n
    all_states = [padded_binary(i, n) for i in range(num_states)]
    
    # print(f"Total number of states: {num_states}")
    # print("Valid single-step transitions:")

    valid_difference_vectors = set()
    
    valid_X_reactions = [] # distributively
    valid_Y_reactions = [] # distributively

    for i in range(num_states):
        for j in range(num_states):
            # Do not consider transitions from a state to itself
            if i == j:
                continue

            if np.sum(np.abs(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i]))) == 1:
                # Determine if it's a phosphorylation or dephosphorylation event
                # A +1 indicates phosphorylation, a -1 indicates dephosphorylation
                element = "X" if np.any(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i]) == 1) else "Y"
                
                if element == "X":
                    print(f"{all_states[i]} --> {all_states[j]} ({element}), {i} -> {j}")
                    valid_X_reactions.append([all_states[i], all_states[j], i, j, element])
                if element == "Y":
                    print(f"{all_states[i]} --> {all_states[j]} ({element}), {i} -> {j}")
                    valid_Y_reactions.append([all_states[i], all_states[j], i, j, element])

                valid_difference_vectors.add(tuple(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i])))

    return valid_X_reactions, valid_Y_reactions

def build_phos_odes(n, prefix=""):
    """
    Build symbolic ODEs for the phosphorylation system for given n (N = 2**n).
    Returns a dict with keys:
      adot, bdot, cdot, xdot, ydot,
    """
    N = 2**n
    a_syms = sp.symbols([f"{prefix}a_{i}" for i in range(N)])
    b_syms = sp.symbols([f"{prefix}b_{i}" for i in range(N)])
    c_syms = sp.symbols([f"{prefix}c_{i}" for i in range(N)])
    x_sym, y_sym = sp.symbols(f"{prefix}x {prefix}y")

    a = sp.Matrix(N, 1, lambda i,j: a_syms[i])
    b = sp.Matrix(N, 1, lambda i,j: b_syms[i])
    c = sp.Matrix(N, 1, lambda i,j: c_syms[i])
    
    x = x_sym
    y = y_sym

    kplus = sp.symbols([f"{prefix}k^+_{i}" for i in range(N)]); kplus[-1] = sp.Integer(0)
    kminus = sp.symbols([f"{prefix}k^-_{i}" for i in range(N)]); kminus[-1] = sp.Integer(0)
    pplus = sp.symbols([f"{prefix}p^+_{i}" for i in range(N)]); pplus[0] = sp.Integer(0)
    pminus = sp.symbols([f"{prefix}p^-_{i}" for i in range(N)]); pminus[0] = sp.Integer(0)

    Kp = sp.diag(*kplus)
    Km = sp.diag(*kminus)
    Pp = sp.diag(*pplus)
    Pm = sp.diag(*pminus)

    # compute allowed transitions
    valid_X_reactions, valid_Y_reactions = calculate_valid_transitions(n)
    
    # Build Alpha and Beta with symbols only at allowed (j,k)
    Alpha = sp.zeros(N, N)
    Beta  = sp.zeros(N, N)
    # create symbols for allowed entries
    for (_, _, j,k, _) in valid_X_reactions:
        Alpha[j,k] = sp.symbols(f"{prefix}alpha_{j}_{k}")
    for (_, _, j,k, _) in valid_Y_reactions:
        Beta[j,k] = sp.symbols(f"{prefix}beta_{j}_{k}")

    ones = sp.Matrix([1]*N)
    Alpha_row_sums = Alpha * ones
    Beta_row_sums = Beta * ones
    DAlpha = sp.diag(*[Alpha_row_sums[i, 0] for i in range(N)])
    DBeta  = sp.diag(*[Beta_row_sums[i, 0] for i in range(N)])
    print(DAlpha)
    print(DBeta)
    adot = Km * b + Pm * c + Alpha.T * b + Beta.T * c - x * (Kp * a) - y * (Pp * a)
    bdot = x * (Kp * a) - Km * b - DAlpha * b
    cdot = y * (Pp * a) - Pm * c - DBeta * c
    bdot[-1] = sp.Integer(0)
    cdot[0] = sp.Integer(0)
    kplus_vec = sp.Matrix(kplus)
    pplus_vec = sp.Matrix(pplus)

    xdot = - x * (kplus_vec.T * a)[0] + (ones.T * (Km + DAlpha) * b)[0]
    ydot = - y * (pplus_vec.T * a)[0] + (ones.T * (Pm + DBeta) * c)[0]
    # print(type(adot[0]))

    return {
        "adot": sp.expand(adot), "bdot": sp.expand(bdot), "cdot": sp.expand(cdot),
        "xdot": sp.expand(xdot), "ydot": sp.expand(ydot),
    }

# Example: build symbolic ODEs for n=2 (N=4)
n = 2
odes = build_phos_odes(n)
odes["cdot"][1]
# sp.pprint(odes["adot"])
# sp.pprint(odes["bdot"])
# sp.pprint(odes["cdot"])
# sp.pprint(odes["xdot"])
# sp.pprint(odes["ydot"])

00 --> 01 (X), 0 -> 1
00 --> 10 (X), 0 -> 2
01 --> 00 (Y), 1 -> 0
01 --> 11 (X), 1 -> 3
10 --> 00 (Y), 2 -> 0
10 --> 11 (X), 2 -> 3
11 --> 01 (Y), 3 -> 1
11 --> 10 (Y), 3 -> 2
Matrix([[alpha_0_1 + alpha_0_2, 0, 0, 0], [0, alpha_1_3, 0, 0], [0, 0, alpha_2_3, 0], [0, 0, 0, 0]])
Matrix([[0, 0, 0, 0], [0, beta_1_0, 0, 0], [0, 0, beta_2_0, 0], [0, 0, 0, beta_3_1 + beta_3_2]])


a_1*p^+_1*y - beta_1_0*c_1 - c_1*p^-_1

In [12]:
odes["cdot"][2]


a_2*p^+_2*y - beta_2_0*c_2 - c_2*p^-_2

In [13]:
odes["cdot"][3]


a_3*p^+_3*y - beta_3_1*c_3 - beta_3_2*c_3 - c_3*p^-_3

In [14]:
odes["xdot"]


-a_0*k^+_0*x - a_1*k^+_1*x - a_2*k^+_2*x + alpha_0_1*b_0 + alpha_0_2*b_0 + alpha_1_3*b_1 + alpha_2_3*b_2 + b_0*k^-_0 + b_1*k^-_1 + b_2*k^-_2

# PURELY SYMBOLIC:

In [15]:
prefix=""
n = 1
N = 2**n
a_syms = sp.symbols([f"{prefix}a_{i}" for i in range(N)])

# gen = all_parameter_generation(n, "distributive", "gamma", (1, 2), verbose = False)
# alpha_matrix = gen.alpha_parameter_generation()
# beta_matrix = gen.beta_parameter_generation()
# k_positive_rates, k_negative_rates = gen.k_parameter_generation()
# p_positive_rates, p_negative_rates = gen.p_parameter_generation()
# k_positive_rates = np.ones(N - 1); k_negative_rates = np.ones(N - 1)
# p_positive_rates = np.ones(N - 1); p_negative_rates = np.ones(N - 1)

# print(k_positive_rates)

x_tot_value = 5
y_tot_value = 4
x_tot = sp.Float(x_tot_value); y_tot = sp.Float(y_tot_value)

a = sp.Matrix(N, 1, lambda i,j: a_syms[i])

ones_vec = sp.ones(1, N - 1)

kplus = sp.symbols([f"{prefix}k^+_{i}" for i in range(N - 1)])
kminus = sp.symbols([f"{prefix}k^-_{i}" for i in range(N - 1)])
pplus = sp.symbols([f"{prefix}p^+_{i}" for i in range(1, N)])
pminus = sp.symbols([f"{prefix}p^-_{i}" for i in range(1, N)])

Kp = sp.diag(*kplus, 0)
# Kp = sp.diag(*[sp.Float(i) for i in k_positive_rates], sp.Float(0))

Km = sp.diag(*[sp.Float(i) for i in k_negative_rates]); Km = Km.row_insert(len(k_negative_rates), sp.zeros(1, len(k_negative_rates)))
# Km_small = sp.diag(*[sp.Float(i) for i in k_negative_rates])
# Km = Km_small.row_insert(len(k_negative_rates), sp.zeros(1, len(k_negative_rates)))  # becomes N x (N-1)

Pp = sp.diag(*pplus, 0)
# Pp = sp.diag(*[sp.Float(i) for i in p_positive_rates], sp.Float(0))

Pm = sp.diag(*[sp.Float(i) for i in p_negative_rates]); Pm = Pm.row_insert(len(p_negative_rates), sp.zeros(1, len(p_negative_rates)))

# Pm_small = sp.diag(*[sp.Float(i) for i in p_negative_rates])
# Pm = Pm_small.row_insert(len(p_negative_rates), sp.zeros(1, len(p_negative_rates)))  # becomes N x (N-1)

valid_X_reactions, valid_Y_reactions = calculate_valid_transitions(n)

Alpha_adjusted = np.delete(alpha_matrix, -1, axis = 0)
Beta_adjusted  = np.delete(beta_matrix, 0, axis = 0)

ones = sp.Matrix([1]*N)
Alpha_row_sums = alpha_matrix * ones
Beta_row_sums = beta_matrix * ones
DAlpha = sp.diag(*[Alpha_row_sums[i, 0] for i in range(N - 1)])
DBeta  = sp.diag(*[Beta_row_sums[i, 0] for i in range(N - 1)])
U = sp.diag(*[sp.Float(i) for i in k_negative_rates])
I = sp.diag(*[sp.Float(i) for i in p_negative_rates])
Q = sp.diag(*[sp.Float(i) for i in k_positive_rates]); Q = Q.col_insert(len(k_positive_rates), sp.zeros(len(k_positive_rates), 1))
D = sp.diag(*[sp.Float(i) for i in p_positive_rates]); D = D.col_insert(0, sp.zeros(len(p_positive_rates), 1))

M = U + DAlpha
N = I + DBeta
G = Km + Alpha_adjusted.T
H = Pm + Beta_adjusted.T
M_inv = M.inv(); N_inv = N.inv()

L1 = G * M_inv * Q - Kp; L2 = H * N_inv * D - Pp
W1 = M_inv * Q; W2 = N_inv * D
a_dot = x_tot * L1 * (1 / (1 + (ones_vec * W1 * a)[0])) * a + y_tot * L2 * (1 / (1 + (ones_vec * W2 * a)[0])) * a
polynomials = sp.simplify(a_dot * (1 + (ones_vec * W1 * a)[0]) * (1 + (ones_vec * W2 * a)[0]))
polynomials
# from sympy import solve_triangulated
# solve_triangulated(polynomials, *a_syms)

# 'lex' or 'grevlex' or 'grlex'
groebner_basis = sp.groebner(polynomials, *a_syms, order = 'lex')
groebner_basis
# roots = sp.solve(groebner_basis, *a_syms, dict = True)
# roots

NameError: name 'k_negative_rates' is not defined

# Computing adjusted matrices from piece of paper:

In [ ]:
from numpy import linalg as LA
import numpy as np
# from julia import Julia
# jl = Julia(compiled_modules=False)
# from julia import Main
import sympy as sp
from itertools import product
from scipy.optimize import root
from scipy.optimize import least_squares
# Initialize HomotopyContinuation in Julia
# Main.eval("using HomotopyContinuation")
# from julia import Julia
# jl = Julia(compiled_modules=False)
# from julia import Main
import numpy as np

# Load HomotopyContinuation once
# Main.eval("using HomotopyContinuation")
import numpy as np


def polynomial_finder(n, a_tot_value, x_tot_value, y_tot_value, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates):

    N = 2**n
    
    x_tot = sp.Float(x_tot_value); y_tot = sp.Float(y_tot_value); a_tot = sp.Float(a_tot_value)

    ones_vec = np.ones(N - 1)
    # ones_vec = np.ones((N-1, 1))
    Kp = np.diag(np.append(k_positive_rates, 0))
    Km = np.append(np.diag(k_negative_rates), np.zeros((1, len(k_negative_rates))), axis=0)

    Pp = np.diag(np.insert(p_positive_rates, 0, 0))
    Pm = np.vstack([np.zeros((1, len(p_negative_rates))), np.diag(p_negative_rates)])

    adjusted_alpha_mat = np.delete(alpha_matrix, -1, axis = 0)
    adjusted_beta_mat = np.delete(beta_matrix, 0, axis = 0)


    Da = np.diag(alpha_matrix[:-1, 1:] @ ones_vec)
    Db = np.diag(beta_matrix[1:, :-1] @ ones_vec)


    U = np.diag(k_negative_rates)
    I = np.diag(p_negative_rates)
    Q = Kp[:-1, :]
    D = np.delete(Pp, 0, axis=0)
    M = U + Da
    N = I + Db

    G = Km + adjusted_alpha_mat.T
    H = Pm + adjusted_beta_mat.T
    M_inv = np.linalg.inv(M); N_inv = np.linalg.inv(N)

    L1 = G @ M_inv @ Q - Kp; L2 = H @ N_inv @ D - Pp
    W1 = M_inv @ Q; W2 = N_inv @ D

    ####### RESCALING #######
    k_neg_0 = k_negative_rates[0]
    # t --> t * k_neg_0 * y_tot / a_tot
    # L1 = L1 * k_neg_0 / a_tot; L2 = L2 * k_neg_0 / a_tot
    # W1 = W1 / a_tot; W2 = W2 / a_tot

    return L1, L2, W1, W2, k_neg_0


from itertools import product

def stability_calculator(a_fixed_points, p, L1, L2, W1, W2):
    N = len(a_fixed_points)
    ones_vec_j = np.ones((1, N-1))  # shape (1, N-1)
    a_fixed_points = np.array(a_fixed_points).reshape((N, 1))  # shape (N, 1)

    L1 = np.array(L1, dtype=float)
    L2 = np.array(L2, dtype=float)
    W1 = np.array(W1, dtype=float)
    W2 = np.array(W2, dtype=float)

    # Compute denominators
    denom1 = 1 + float(ones_vec_j @ W1 @ a_fixed_points)
    denom2 = 1 + float(ones_vec_j @ W2 @ a_fixed_points)

    # Compute terms
    term1 = L1 / denom1 - ((L1 @ a_fixed_points @ ones_vec_j @ W1) / denom1**2)
    term2 = L2 / denom2 - ((L2 @ a_fixed_points @ ones_vec_j @ W2) / denom2**2)
    J = p * term1 - term2

    eigenvalues = LA.eigvals(J)
    real_parts = np.real(eigenvalues)
    # print(real_parts)
    return real_parts


def stable_fp_checker(a_tot_value, x_tot_value, y_tot_value,
                      alpha_matrix, beta_matrix,
                      k_positive_rates, k_negative_rates,
                      p_positive_rates, p_negative_rates):
    n = 2
    N = 2**n

    # Unknowns: a0, a1, ..., a_{N-2} (last one is from conservation)
    a_syms = sp.symbols([f"a{i}" for i in range(N - 1)], real=True)
    a_syms_reduced = sp.Matrix(list(a_syms))
    a_last_expr = 1 - sum(a_syms)
    a_full = sp.Matrix([a_syms[i] if i < N-1 else a_last_expr for i in range(N)])
    print("a_full:", a_full)
    # a_red = sp.symbols(f"a0:{N-1}", real=True)
    # a_full_syms = list(a_red) + [sp.Symbol(f"a{N-1}", real=True)]
    a_tot_sym, x_tot_sym, y_tot_sym = sp.symbols("a_tot_sym x_tot_sym y_tot_sym", real=True)
    a_vec_sym = sp.Matrix(a_syms)
    print("a_vec_sym:", a_vec_sym)
    # Compute L1, L2, W1, W2
    L1, L2, W1, W2, k = polynomial_finder(
        n, a_tot_value, x_tot_value, y_tot_value,
        alpha_matrix, beta_matrix,
        k_positive_rates, k_negative_rates,
        p_positive_rates, p_negative_rates
    )
    
    L1 = np.array(L1, dtype=float)
    L2 = np.array(L2, dtype=float)
    W1 = np.array(W1, dtype=float)
    W2 = np.array(W2, dtype=float)

    ###### RESCALING
    L1 = L1 * (k / a_tot_value); L2 = L2 * (k / a_tot_value)
    W1 = W1 * a_tot_value; W2 = W2 * a_tot_value
    ######

    ##### NORMALIZNG A VECTOR 
    a_dim_sym = a_full * a_tot_sym
    print("a_dim_sym:", a_dim_sym)

    # a_reduced_dim = sp.Matrix([a_dim_sym[i, 0] for i in range(N - 1)])
    # print("a_reduced_dim:", a_reduced_dim)

    L1_sym = sp.Matrix(L1.tolist())
    L2_sym = sp.Matrix(L2.tolist())
    W1_sym = sp.Matrix(W1.tolist())
    W2_sym = sp.Matrix(W1.tolist())

    ones_vec_sym = sp.Matrix([[1] * W2_sym.rows])

    x_tot = sp.Float(x_tot_value); y_tot = sp.Float(y_tot_value) 

    inner_W1 = (ones_vec_sym * W1_sym * a_dim_sym)[0, 0]
    inner_W2 = (ones_vec_sym * W2_sym * a_dim_sym)[0, 0]
    # print(inner_W1)
    L1a = L1_sym * a_dim_sym
    L2a = L2_sym * a_dim_sym
    p = x_tot_sym / y_tot_sym
    poly_exprs = p * L1a * (1 + inner_W2) + L2a * (1 + inner_W1)   
    
    # poly_exprs.append(sp.simplify(1 - sum(a_syms)))

    # print(poly_exprs)

    polynomials_list = [sp.simplify(poly_exprs[i, 0]) for i in range(N - 1)]
    # print(sp.Matrix([a_vec_sym[i, 0] for i in range(N - 1)]))

    # sum_reduced = sum(a_red)
    # subs_dict = {a_full_syms[-1]: a_tot - sum_reduced}

    # reduced F: take first N-1 components and substitute
    # polynomials_reduced = [sp.simplify(poly_exprs[i,0].subs(subs_dict)) for i in range(N-1)]
    # Add conservation equation for a_tot
    polynomials_list.append(sp.simplify(sum(a_syms) - 1))
    # print(polynomials_list)
    subs_numeric = {a_tot_sym: float(a_tot_value), x_tot_sym: float(x_tot_value), y_tot_sym: float(y_tot_value)}

    polynomials_numeric = [sp.N(p.subs(subs_numeric)) for p in polynomials_list]
    print(polynomials_numeric)
    print(len(polynomials_numeric))
    # Initial guess
    x0 = [a_tot_value / N] * (N - 1)  # N variables (a_0, ..., a_{N-1})

    # Variables to solve for: a_full_syms
    try:
        sol = sp.nsolve(polynomials_numeric, list(a_syms), x0, tol=1e-2, maxsteps=200)
    except Exception as e:
        print("nsolve failed:", e)
        return []  # failed
    # Recover full solution including a_N
    sol = np.asarray(list(sol))
    print("sol:", sol)

    a_full_sol_num = np.zeros((N,))
    a_full_sol_num[:N-1] = sol
    a_full_sol_num[N-1] = 1 - float(np.sum(sol))
    a_solution = (a_full_sol_num * float(a_tot_value)).reshape((N, 1))
    print(a_solution)
    ones_row = np.ones((1, W1.shape[0]))
    x_solution = float(x_tot_value) / (1.0 + float(ones_row @ (W1 @ a_solution)))
    y_solution = float(y_tot_value) / (1.0 + float(ones_row @ (W2 @ a_solution)))
    b_solution = x_solution * (W1 @ a_solution).flatten()
    c_solution = y_solution * (W2 @ a_solution).flatten()

    tol = 1e-1
    eigenvalues = stability_calculator(a_solution.flatten(), x_tot_value / y_tot_value, L1, L2, W1, W2)
    print(x_solution + np.sum(b_solution), x_tot_value)
    print(y_solution + np.sum(c_solution), y_tot_value)
    print(np.sum(a_solution) + np.sum(b_solution) + np.sum(c_solution), a_tot_value)

    if (
        np.all(a_solution > 0)
        and np.all(eigenvalues < 0)
        and np.isclose(x_solution + np.sum(b_solution), x_tot_value, tol)
        and np.isclose(y_solution + np.sum(c_solution), y_tot_value, tol)
        and np.isclose(np.sum(a_solution) + np.sum(b_solution) + np.sum(c_solution), a_tot_value, tol)
    ):
        print("Positivity confirmed, conservation confirmed, and stability confirmed!")
        return [a_solution.flatten()]
    else:
        print("Fixed point is not stable")
        return []

    # Recover full solution including a_N
    # a_solution = np.asarray(list(sol.x) + [a_tot_value - sum(sol.x)])
    # a_col = np.asarray(a_solution, dtype=float).reshape((N,1))
    # ones_row = np.ones((1,N-1))
    # x_solution = float(x_tot_value) / (1.0 + float(ones_row @ (W1 @ a_col)))
    # y_solution = float(y_tot_value) / (1.0 + float(ones_row @ (W2 @ a_col)))
    # b_solution = x_solution * (W1 @ a_col).flatten()
    # c_solution = y_solution * (W2 @ a_col).flatten()

    # print("a_solution:", a_solution)
    # print("x_solution:", x_solution)
    # print("y_solution:", y_solution)
    # print("b_solution:", b_solution)
    # print("c_solution:", c_solution)
    # print(np.sum(a_solution))
    # tol = 1e-4
    # eigenvalues = stability_calculator(a_solution, x_tot_value / y_tot_value, L1, L2, W1, W2)
    # print(x_solution + np.sum(b_solution), x_tot_value)
    # print(y_solution + np.sum(c_solution), y_tot_value)
    # print(np.sum(a_solution) + np.sum(b_solution) + np.sum(c_solution), a_tot_value)

    # if (
    #     np.all(a_solution > 0)
    #     and np.all(eigenvalues < 0)
    #     and np.isclose(x_solution + np.sum(b_solution), x_tot_value, tol)
    #     and np.isclose(y_solution + np.sum(c_solution), y_tot_value, tol)
    #     and np.isclose(np.sum(a_solution) + np.sum(b_solution) + np.sum(c_solution), a_tot_value, tol)
    # ):
    #     print("Positivity confirmed, conservation confirmed, and stability confirmed!")
    #     return [a_solution]
    # else:
    #     print("Fixed point is not stable")
    #     return []
    
def log_uniform_sample(a: float, b: float, n: int, base: float = np.e) -> np.ndarray:
    """
    Sample n points uniformly in logarithmic space from range (a, b).

    Args:
        a (float): Lower bound (must be > 0).
        b (float): Upper bound (must be > a).
        n (int): Number of samples to draw.
        base (float): Logarithmic base (default: natural log).

    Returns:
        np.ndarray: Array of shape (n,) with log-uniformly sampled values.
    """
    if a <= 0 or b <= 0:
        raise ValueError("a and b must be positive.")
    if a >= b:
        raise ValueError("a must be smaller than b.")

    log_a = np.log(a) / np.log(base)
    log_b = np.log(b) / np.log(base)
    log_samples = np.random.uniform(log_a, log_b, n)
    samples = base ** log_samples
    return samples

def simulation(simulation_size):
    a_tot_value = 100
    x_tot_value_parameter_array = np.array([log_uniform_sample(1e-4, 1e-1, n=1, base=np.e) for _ in range(simulation_size)])
    y_tot_value_parameter_array = np.array([log_uniform_sample(1e-4, 1e-1, n=1, base=np.e) for _ in range(simulation_size)])
    alpha_matrix_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=4, base=np.e) for _ in range(simulation_size)])
    beta_matrix_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=4, base=np.e) for _ in range(simulation_size)])
    k_positive_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])
    k_negative_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])
    p_positive_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])
    p_negative_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])

    final_array = np.empty((0, 27), dtype=float) 

    for i in range(simulation_size):
    
        x_tot_value = x_tot_value_parameter_array[i][0]
        y_tot_value = y_tot_value_parameter_array[i][0]

        alpha_matrix = np.array([
            [0, alpha_matrix_parameter_array[i][0], alpha_matrix_parameter_array[i][1], 0],
            [0, 0, 0, alpha_matrix_parameter_array[i][2]],
            [0, 0, 0, alpha_matrix_parameter_array[i][3]],
            [0, 0, 0, 0]
        ])

        beta_matrix = np.array([
            [0, 0, 0, 0],
            [beta_matrix_parameter_array[i][0], 0, 0, 0],
            [beta_matrix_parameter_array[i][1], 0, 0, 0],
            [0, beta_matrix_parameter_array[i][2], beta_matrix_parameter_array[i][3], 0]
        ])

        k_positive_rates = k_positive_parameter_array[i]
        k_negative_rates = k_negative_parameter_array[i]
        p_positive_rates = p_positive_parameter_array[i]
        p_negative_rates = p_negative_parameter_array[i]

        stable_fp_array = stable_fp_checker(a_tot_value, x_tot_value, y_tot_value, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates)
        print(stable_fp_array)
        # Flatten stable_fp_array to 1D if it is not empty, else fill with NaNs
        if len(stable_fp_array) > 0:
            stable_fp_flat = np.ravel(stable_fp_array)
            array = np.concatenate([stable_fp_flat, 
                                   np.array([a_tot_value]), 
                                   np.array([x_tot_value]), 
                                   np.array([y_tot_value]),
                                   np.ravel(alpha_matrix_parameter_array[i]), 
                                   np.ravel(beta_matrix_parameter_array[i]), 
                                   np.ravel(k_positive_parameter_array[i]), 
                                   np.ravel(k_negative_parameter_array[i]), 
                                   np.ravel(p_positive_parameter_array[i]), 
                                   np.ravel(p_negative_parameter_array[i])])
            final_array = np.append(final_array, [array], axis = 0)

    header_string = "a_0_stable_fp, a_1_stable_fp, a_2_stable_fp, a_3_stable_fp, a_tot, x_tot, y_tot, alpha_matrix_parameters, beta_matrix_parameters, k_+_parameters, k_-_parameters, p_+_parameters, p_-_parameters"
    filename = f"test_{simulation_size}.csv"
    np.savetxt(filename, final_array, delimiter=",", header=header_string, comments='')
    return 

simulation(500)


a_full: Matrix([[a0], [a1], [a2], [-a0 - a1 - a2 + 1]])
a_vec_sym: Matrix([[a0], [a1], [a2]])
a_dim_sym: Matrix([[a0*a_tot_sym], [a1*a_tot_sym], [a2*a_tot_sym], [a_tot_sym*(-a0 - a1 - a2 + 1)]])
[5292.38963303759*(-0.0181536856593769*a0 + 0.0350664070904007*a1 + 0.00480521249012753*a2)*(47.4617502626769*a0 + 81613.7363427407*a1 + 2.1725347348624*a2 + 1.0), 5292.38963303759*(0.00857613406899399*a0 - 0.181313061218661*a1 - 0.00235609694376844*a2 + 0.00235609694376844)*(47.4617502626769*a0 + 81613.7363427407*a1 + 2.1725347348624*a2 + 1.0), 5292.38963303759*(-0.114997543069128*a0 - 0.122218997715742*a1 - 0.127032081380324*a2 + 0.122218997715742)*(47.4617502626769*a0 + 81613.7363427407*a1 + 2.1725347348624*a2 + 1.0), a0 + a1 + a2 - 1.0]
4
nsolve failed: Could not find root within given tolerance. (0.000259727979285385356976 > 1e-05)
Try another starting point or tweak arguments.
[]
a_full: Matrix([[a0], [a1], [a2], [-a0 - a1 - a2 + 1]])
a_vec_sym: Matrix([[a0], [a1], [a2]])
a_dim_sym: Matri

/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_60389/2119138580.py:193: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_solution = float(x_tot_value) / (1.0 + float(ones_row @ (W1 @ a_solution)))
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_60389/2119138580.py:194: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_solution = float(y_tot_value) / (1.0 + float(ones_row @ (W2 @ a_solution)))
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_60389/2119138580.py:80: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element f

[80102.7809751515*(-7.45841074502847e-6*a0 + 7.78430123736981e-5*a1 + 0.000277003078926176*a2)*(459.487532450695*a0 + 4534.54274348404*a1 + 30802.1643096981*a2 + 1.0), 80102.7809751515*(-2.3758240076986e-5*a0 - 0.000116246148824992*a1 - 3.12136281394013e-5*a2 + 3.12136281394013e-5)*(459.487532450695*a0 + 4534.54274348404*a1 + 30802.1643096981*a2 + 1.0), 80102.7809751515*(-1.39465169880733e-5*a0 - 1.39495396706865e-5*a1 - 0.000300576549716008*a2 + 1.39495396706865e-5)*(459.487532450695*a0 + 4534.54274348404*a1 + 30802.1643096981*a2 + 1.0), a0 + a1 + a2 - 1.0]
4
nsolve failed: Could not find root within given tolerance. (0.000182752963460037606892 > 1e-05)
Try another starting point or tweak arguments.
[]
a_full: Matrix([[a0], [a1], [a2], [-a0 - a1 - a2 + 1]])
a_vec_sym: Matrix([[a0], [a1], [a2]])
a_dim_sym: Matrix([[a0*a_tot_sym], [a1*a_tot_sym], [a2*a_tot_sym], [a_tot_sym*(-a0 - a1 - a2 + 1)]])
[18415.8289069154*(-0.0407022119034557*a0 + 2.85621320613328*a1 + 0.133043420160955*a2)*(65.

In [58]:
def stable_fp_normalized(a_tot_value, x_tot_value, y_tot_value,
                         alpha_matrix, beta_matrix,
                         k_positive_rates, k_negative_rates,
                         p_positive_rates, p_negative_rates,
                         max_tries: int = 8):
    """
    Solve for normalized s_i = a_i / a_tot. Returns dimensional a vector if found & stable, else [].
    """
    n = 2
    N = 2**n

    # unknown normalized symbols s0..s_{N-2}
    s_syms = sp.symbols(f's0:{N-1}', real=True)           # length N-1
    s_reduced_vec = sp.Matrix(list(s_syms))              # (N-1)x1
    s_last_expr = 1 - sum(s_syms)
    s_full = sp.Matrix([s_syms[i] if i < N-1 else s_last_expr for i in range(N)])  # N x 1

    # parameter symbols (we will subs numeric values)
    a_tot_sym, x_tot_sym, y_tot_sym = sp.symbols('a_tot_sym x_tot_sym y_tot_sym', real=True)

    # compute numeric L,W matrices
    L1_num, L2_num, W1_num, W2_num, k_ref = polynomial_finder(
        n, a_tot_value, x_tot_value, y_tot_value,
        alpha_matrix, beta_matrix,
        k_positive_rates, k_negative_rates,
        p_positive_rates, p_negative_rates
    )
    if L1_num is None:
        return []

    L1 = np.array(L1_num, dtype=float)
    L2 = np.array(L2_num, dtype=float)
    W1 = np.array(W1_num, dtype=float)
    W2 = np.array(W2_num, dtype=float)

    # apply your chosen rescaling (keep consistent)
    L1 = L1 * (k_ref / a_tot_value)
    L2 = L2 * (k_ref / a_tot_value)
    W1 = W1 * a_tot_value
    W2 = W2 * a_tot_value

    # convert to sympy matrices
    L1_sym = sp.Matrix(L1.tolist())
    L2_sym = sp.Matrix(L2.tolist())
    W1_sym = sp.Matrix(W1.tolist())
    W2_sym = sp.Matrix(W2.tolist())

    # Build dimensional a vector symbolically: a = s_full * a_tot_sym  (N x 1)
    a_dim_vec = s_full * a_tot_sym

    # Build ones-row compatible with W1_sym/W2_sym row count (robust)
    ones_row_W1 = sp.Matrix([[1] * W1_sym.rows])   # 1 x rows(W1)
    ones_row_W2 = sp.Matrix([[1] * W2_sym.rows])

    # Sanity assert: W1 rows should equal W2 rows (they typically are both N-1)
    if W1_sym.rows != W2_sym.rows:
        print("Warning: W1 and W2 have different number of rows:", W1_sym.rows, W2_sym.rows)

    # inner products: use full a_dim_vec (N x 1) because W1_sym is (rows x N)
    inner_W1 = (ones_row_W1 * W1_sym * a_dim_vec)[0, 0]
    inner_W2 = (ones_row_W2 * W2_sym * a_dim_vec)[0, 0]

    # numerators
    L1a = L1_sym * a_dim_vec   # N x 1
    L2a = L2_sym * a_dim_vec   # N x 1

    p_sym = x_tot_sym / y_tot_sym

    # cleared denominators: p*L1a*(1+u2) + L2a*(1+u1) = 0
    poly_exprs = p_sym * (L1a * (1 + inner_W2)) + (L2a * (1 + inner_W1))

    # reduced system: first N-1 component equations (these depend on s_syms)
    polys_symbolic = [sp.simplify(poly_exprs[i, 0]) for i in range(N - 1)]
    # conservation in normalized variables
    polys_symbolic.append(sp.simplify(sum(s_syms) - 1))

    # substitute numeric parameters once
    subs_numeric = {a_tot_sym: float(a_tot_value), x_tot_sym: float(x_tot_value), y_tot_sym: float(y_tot_value)}
    polys_numeric = [sp.N(p.subs(subs_numeric)) for p in polys_symbolic]

    # Solve with nsolve for s_syms (N-1 unknowns)
    from sympy import nsolve
    sol_s = None
    for attempt in range(max_tries):
        if attempt == 0:
            guess = [1.0 / N] * (N - 1)
        else:
            r = np.random.rand(N - 1)
            r = r / r.sum()
            guess = r.tolist()

        try:
            sol_sym = nsolve(polys_numeric, list(s_syms), guess, tol=1e-13, maxsteps=200)
            sol_list = [complex(sp.N(sv)) for sv in sol_sym]
            # accept real and in [0,1] (with tiny tolerance)
            if all(abs(z.imag) < 1e-8 and -1e-9 <= z.real <= 1+1e-9 for z in sol_list):
                sol_s = np.array([float(z.real) for z in sol_list])
                break
        except Exception:
            continue

    if sol_s is None:
        print("nsolve failed to find a normalized solution after tries.")
        return []

    # build full normalized solution and dimensional a_solution
    s_full_num = np.zeros((N,))
    s_full_num[:N-1] = sol_s
    s_full_num[N-1] = 1.0 - float(np.sum(sol_s))
    a_solution = (s_full_num * float(a_tot_value)).reshape((N, 1))

    # compute x,y,b,c as before (note: W1 shape rows x N, use first rows accordingly)
    ones_row_np = np.ones((1, W1.shape[0]))  # numeric ones row for W1/W2
    x_solution = float(x_tot_value) / (1.0 + float(ones_row_np @ (W1 @ a_solution)))
    y_solution = float(y_tot_value) / (1.0 + float(ones_row_np @ (W2 @ a_solution)))
    b_solution = x_solution * (W1 @ a_solution).flatten()
    c_solution = y_solution * (W2 @ a_solution).flatten()

    # stability check
    tol = 1e-1
    eigenvalues = stability_calculator(a_solution.flatten(), x_tot_value / y_tot_value, L1, L2, W1, W2)

    print("x check:", x_solution + np.sum(b_solution), x_tot_value)
    print("y check:", y_solution + np.sum(c_solution), y_tot_value)
    print("total check:", np.sum(a_solution) + np.sum(b_solution) + np.sum(c_solution), a_tot_value)

    if (
        np.all(a_solution > -tol)
        and np.max(np.real(eigenvalues)) < -1e-2
        and np.isclose(x_solution + np.sum(b_solution), x_tot_value, atol=1e-1)
        and np.isclose(y_solution + np.sum(c_solution), y_tot_value, atol=1e-1)
        and np.isclose(np.sum(a_solution) + np.sum(b_solution) + np.sum(c_solution), a_tot_value, atol=1e-4)
    ):
        print("Normalized solution found: positivity, conservation, and stability confirmed!")
        return [a_solution.flatten()]
    else:
        print("Solution found but failed positivity/conservation/stability tests.")
        return []


def simulation(simulation_size):
    a_tot_value = 10
    x_tot_value_parameter_array = np.array([log_uniform_sample(1e-4, 1e-1, n=1, base=np.e) for _ in range(simulation_size)])
    y_tot_value_parameter_array = np.array([log_uniform_sample(1e-4, 1e-1, n=1, base=np.e) for _ in range(simulation_size)])
    alpha_matrix_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=4, base=np.e) for _ in range(simulation_size)])
    beta_matrix_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=4, base=np.e) for _ in range(simulation_size)])
    k_positive_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])
    k_negative_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])
    p_positive_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])
    p_negative_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])

    final_array = np.empty((0, 27), dtype=float) 

    for i in range(simulation_size):
    
        x_tot_value = x_tot_value_parameter_array[i][0]
        y_tot_value = y_tot_value_parameter_array[i][0]

        alpha_matrix = np.array([
            [0, alpha_matrix_parameter_array[i][0], alpha_matrix_parameter_array[i][1], 0],
            [0, 0, 0, alpha_matrix_parameter_array[i][2]],
            [0, 0, 0, alpha_matrix_parameter_array[i][3]],
            [0, 0, 0, 0]
        ])

        beta_matrix = np.array([
            [0, 0, 0, 0],
            [beta_matrix_parameter_array[i][0], 0, 0, 0],
            [beta_matrix_parameter_array[i][1], 0, 0, 0],
            [0, beta_matrix_parameter_array[i][2], beta_matrix_parameter_array[i][3], 0]
        ])

        k_positive_rates = k_positive_parameter_array[i]
        k_negative_rates = k_negative_parameter_array[i]
        p_positive_rates = p_positive_parameter_array[i]
        p_negative_rates = p_negative_parameter_array[i]

        stable_fp_array = stable_fp_normalized(a_tot_value, x_tot_value, y_tot_value, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates)
        print(stable_fp_array)
        # Flatten stable_fp_array to 1D if it is not empty, else fill with NaNs
        if len(stable_fp_array) > 0:
            stable_fp_flat = np.ravel(stable_fp_array)
            array = np.concatenate([stable_fp_flat, 
                                   np.array([a_tot_value]), 
                                   np.array([x_tot_value]), 
                                   np.array([y_tot_value]),
                                   np.ravel(alpha_matrix_parameter_array[i]), 
                                   np.ravel(beta_matrix_parameter_array[i]), 
                                   np.ravel(k_positive_parameter_array[i]), 
                                   np.ravel(k_negative_parameter_array[i]), 
                                   np.ravel(p_positive_parameter_array[i]), 
                                   np.ravel(p_negative_parameter_array[i])])
            final_array = np.append(final_array, [array], axis = 0)

    header_string = "a_0_stable_fp, a_1_stable_fp, a_2_stable_fp, a_3_stable_fp, a_tot, x_tot, y_tot, alpha_matrix_parameters, beta_matrix_parameters, k_+_parameters, k_-_parameters, p_+_parameters, p_-_parameters"
    filename = f"test_{simulation_size}.csv"
    np.savetxt(filename, final_array, delimiter=",", header=header_string, comments='')
    return 

simulation(50)

nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]


/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_60389/2571037656.py:114: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_solution = float(x_tot_value) / (1.0 + float(ones_row_np @ (W1 @ a_solution)))
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_60389/2571037656.py:115: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_solution = float(y_tot_value) / (1.0 + float(ones_row_np @ (W2 @ a_solution)))
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_60389/1745381038.py:80: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single ele

x check: 0.00012893215009488076 0.00012893215009488078
y check: 0.022148623054991465 0.022148623054991462
total check: 10.000082379806983 10
Solution found but failed positivity/conservation/stability tests.
[]
nsolve failed to find a normalized solution after tries.
[]
x check: 0.00022505494370442572 0.00022505494370442572
y check: 0.0032340645526261693 0.0032340645526261693
total check: 10.00029579873686 10
Solution found but failed positivity/conservation/stability tests.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized solution after tries.
[]
nsolve failed to find a normalized so

In [266]:
groebner_basis = sp.groebner(polynomials, *a_syms)
groebner_basis

GroebnerBasis([1.0*a_0, 1.0*a_1], a_0, a_1, domain='RR', order='lex')

In [264]:
roots = sp.solve(groebner_basis, *a_syms, dict = True, simplify=False)
roots

[{a_0: 0.0, a_1: 0.0}]

In [69]:
import numpy as np

# Initialize an empty 2D array with 0 rows and 3 columns
empty_array = np.empty((0, 3), dtype=int) 
print(empty_array)
# Create the array to append
new_row = np.array([[10, 20, 30]])

# Append the new row
empty_array = np.append(empty_array, new_row, axis=0)

# Append another row
another_row = np.array([[40, 50, 60]])
empty_array = np.append(empty_array, another_row, axis=0)

print(empty_array)

for i in range(len(empty_array)):
    print(empty_array[i])
    if np.all(empty_array[i] > 0):
        print("Positivity confirmed")

[]
[[10 20 30]
 [40 50 60]]
[10 20 30]
Positivity confirmed
[40 50 60]
Positivity confirmed


In [40]:
import numpy as np

def sample_log_uniform(n, a, b, size=1, rng=None):
    """
    Sample points uniformly in log-space from an n-dimensional hypercube.

    Parameters
    ----------
    n : int
        Number of dimensions (hypercube dimensionality).
    a, b : float or array-like of length n
        Lower and upper bounds for each coordinate. Must be > 0.
        If scalar, the same range (a,b) is used for all dimensions.
    size : int
        Number of samples to draw (returns array of shape (size, n)).
    rng : None or np.random.Generator
        Optional numpy random Generator for reproducible draws.

    Returns
    -------
    samples : ndarray, shape (size, n)
        Samples drawn so that log(samples) is uniformly distributed
        in the n-dimensional box [log(a), log(b)]^n.
    """
    if rng is None:
        rng = np.random.default_rng()

    a = np.asarray(a)
    b = np.asarray(b)

    # allow scalar bounds or per-dimension bounds
    if a.shape == () and b.shape == ():
        log_a = np.log(a)
        log_b = np.log(b)
        low = log_a
        high = log_b
    else:
        a = a.reshape(-1)
        b = b.reshape(-1)
        if a.size != n or b.size != n:
            raise ValueError("If a or b are arrays they must have length n.")
        low = np.log(a)
        high = np.log(b)

    if np.any(high <= low):
        raise ValueError("Require b > a (elementwise) and both > 0.")

    # draw uniformly in log-space then exponentiate
    u = rng.uniform(low=low, high=high, size=(size, n))
    return np.exp(u)

# Example 3: reproducible draws
rng = np.random.default_rng(42)
samples = sample_log_uniform(n=3, a=10e-1, b=10e7, size=10, rng=None)
print(samples)

[[5.78304517e+01 1.82956144e+02 2.71619238e+07]
 [2.04273050e+00 1.16511733e+06 1.36519063e+05]
 [1.68285696e+06 2.83459746e+03 3.62775105e+06]
 [6.67450880e+02 9.40138456e+03 5.84308078e+01]
 [1.21057522e+00 1.74155583e+01 5.89683717e+03]
 [2.57308998e+05 1.95258514e+05 2.50466426e+01]
 [2.30474971e+01 3.41773373e+05 1.25218651e+06]
 [5.55052535e+04 1.92117067e+02 3.31439272e+07]
 [8.00624336e+01 2.00289200e+02 5.03063927e+01]
 [3.78813091e+06 7.92873229e+02 1.01604200e+05]]
